In [1]:
import numpy as np
import scipy
import scipy.linalg
import scipy.stats
import matplotlib.pyplot as plt
import math
import sklearn
from sklearn import linear_model
from pandas import Series, DataFrame
import pandas as pd
import sqlite3
from scipy.optimize import minimize
import pandas.io.sql as sql
from IPython.display import display_html
from decimal import Decimal
from decimal import localcontext
from scipy.integrate import quad,dblquad,nquad
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle Code</button>""", raw=True)
%matplotlib inline

Toggle Code

In [2]:
### data unpack

G=1200;n=100;K_true=3

name='exp10'

database=name
con=sqlite3.connect('D:\\Python Database\\FM\\'+database)
B_true=np.array(sql.read_sql("SELECT * from B",con,index_col='index'))
Omega_true=np.array(sql.read_sql("SELECT * from Omega",con,index_col='index'))
Y_true=np.array([np.array(sql.read_sql("SELECT * from Y"+str(i),con,index_col='index')['0']) for i in range(n)]).T
con.close()

In [3]:
### sampler

def update_B():
    global B
    
    for j in range(G):
        Dj_inv=np.diag([phi[j,k]*tau[k] for k in range(K)])
        B_var=np.linalg.inv(Dj_inv+sigma_inv2[j]*Omega.dot(Omega.T))
        B_mean=B_var.dot(Omega).dot(Y[j,:])*sigma_inv2[j]
        B[j,:]=scipy.stats.multivariate_normal.rvs(B_mean,B_var)
    return

def update_sigma():
    global sigma_inv2
    
    sigma_inv2=np.array([scipy.stats.gamma.rvs(a_sigma+n/2)/(b_sigma+np.sum((Y[j,:]-B[j,:].dot(Omega))**2)/2) for j in range(G)])
    return

def update_Omega():  ### for normal factor
    global Omega
    
    Omega_var=np.linalg.inv(np.eye(K)+np.multiply(B.T,sigma_inv2).dot(B))
    Omega_mean=Omega_var.dot(np.multiply(B.T,sigma_inv2).dot(Y))
    Omega_var_half=np.linalg.cholesky(Omega_var)
    Omega=Omega_mean+Omega_var_half.dot(np.random.normal(size=(K,n)))
    return

def update_Omega5():  ### for orthogonal factor
    global Omega

    def dlog_p(d):
        return (n-K-2)/2*np.log(n-d**2)+w*d/s2
    
    Omega=Omega.T
    Sigma=1/sigma_inv2

    for k in range(K):
        s2=1/sum(B[:,k]*(1/Sigma)*B[:,k])
        if s2==np.inf:
            A=scipy.zeros((n,n))
            A[:,0:K-1]=np.delete(Omega,k,1 )
            nor=scipy.stats.multivariate_normal.rvs(mean=scipy.zeros(n-K+1))
            Omega[:,k]=np.sqrt(n)/np.sqrt(sum(nor**2))*nor.dot(Q.T[K-1:])
            continue

        mean=(Y.T-Omega.dot(B.T)+np.outer(Omega[:,k],B[:,k])).dot(B[:,k]/Sigma)*s2
        Omega_k_=np.delete(Omega.T, k, 0)
        proj1=Omega_k_.T.dot(scipy.linalg.inv(Omega_k_.dot(Omega_k_.T))).dot(Omega_k_)
        Omega_kw=Omega.T.copy()
        Omega_kw[k]=mean
        proj_mean=mean-mean.dot(proj1)
        w=np.sqrt(sum(proj_mean**2))

        ini=(-(n-K-2)*s2+np.sqrt((n-K-2)**2*s2**2+4*w**2*n))/(2*w)
        std=abs(n-ini**2)/np.sqrt((n+ini**2)*(n-K-2))
        d=ini
        for metro_itr in range(100):
            d_proposal=d+scipy.stats.norm.rvs()*std/2
            if d_proposal>-np.sqrt(n) and d_proposal<np.sqrt(n):
                if np.log(scipy.stats.uniform.rvs())<dlog_p(d_proposal)-dlog_p(d):
                    d=d_proposal
        A=scipy.zeros((n,n))
        A[:,0:K]=Omega_kw.T
        Q, R = scipy.linalg.qr(A)
        nor=scipy.stats.multivariate_normal.rvs(mean=scipy.zeros(n-K))
        Omega[:,k]=d*proj_mean/w+np.sqrt(n-d**2)/np.sqrt(sum(nor**2))*nor.dot(Q.T[K:])
        
    Omega=Omega.T
    return

def update_phi():  
    global phi
    
    phi=np.array([[scipy.stats.gamma.rvs((v+1)/2)/((v+tau[k]*B[j,k]**2)/2) for k in range(K)] for j in range(G)])
    return 

def update_delta():
    global delta,tau
    
    def tau_lh(l,h):
        if l==0 and h==0:
            return 1
        else:
            #print(delta,l,h,np.prod(delta[:l])/delta[h])
            return np.prod(delta[:l+1])/delta[h]
        
    delta[0]=scipy.stats.gamma.rvs(a1+G*K/2)/(1+np.sum([tau_lh(l,0)*np.sum([phi[j,l]*B[j,l]**2 for j in range(G)]) for l in range(K)])/2)
    for h in range(1,K):
        delta[h]=scipy.stats.gamma.rvs(a2+G*(K-h)/2)/(1+np.sum([tau_lh(l,h)*np.sum([phi[j,l]*B[j,l]**2 for j in range(G)]) for l in range(h,K)])/2)
    tau=np.array([np.prod(delta[:k+1]) for k in range(K)])
    
    return

def update_a():
    global a1,a2
    
    def log_pa1(x):
        return -np.log(scipy.special.gamma(x))+np.log(x)+(x-1)*np.log(delta[0])-x
    def log_pa2(x):
        return -(K-1)*np.log(scipy.special.gamma(x))+np.log(x)+(x-1)*(np.sum([np.log(delta[k]) for k in range(1,K)]))-x
    
    res1=minimize(lambda x:-log_pa1(x), a1, method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
    a1_temp=res1.x[0]
    res2=minimize(lambda x:-log_pa2(x), a1, method='nelder-mead',options={'xatol': 1e-8, 'disp': True})
    a2_temp=res2.x[0]
    
    for itr in range(1000):
        proposal_a1=a1_temp+scipy.stats.norm.rvs()
        #print(proposal_a1)
        if proposal_a1<=0:
            continue 
        dif1=log_pa1(proposal_a1)-log_pa1(a1_temp)
        if scipy.stats.uniform.rvs()<np.exp(dif1):
            a1_temp=proposal_a1.copy()
    a1=a1_temp.copy()
    
    for itr in range(1000):
        proposal_a2=a2_temp+scipy.stats.norm.rvs()*0.3
        #print(proposal_a2)
        if proposal_a2<=0:
            continue 
        dif2=log_pa2(proposal_a2)-log_pa2(a2_temp)
        if scipy.stats.uniform.rvs()<np.exp(dif2):
            a2_temp=proposal_a2.copy()
    a2=a2_temp.copy()
            
    return



In [4]:
### envirnment setup

#v=0.5;a_sigma=1;b_sigma=0.3;K=5
v=3;a_sigma=0.5;b_sigma=0.5;K=5

Y=Y_true.copy()
B_start=np.random.normal(size=(G,K))*0.001
B_start[:,:K_true]=B_true.copy()
Omega_start=np.random.normal(size=(K,n))
Omega_start[:K_true,:]=Omega_true.copy()

In [5]:
B=B_start.copy()
Omega=Omega_start.copy()
sigma_inv2=np.ones(G)
phi=np.ones((G,K))
a1=2
a2=2
delta=np.ones(K)
delta[0]=1
tau=np.array([np.prod(delta[:k+1]) for k in range(K)])

database=name+'-Dunsonfm'
con=sqlite3.connect('D:\\Python Database\\FM\\'+database)

for rep in range(3000):
    print(rep)
    
    update_Omega()
    update_B()
    update_sigma()
    update_phi()
    update_a()
    update_delta()
    
    sql.to_sql(DataFrame(Omega),'Omega'+str(rep),con)
    sql.to_sql(DataFrame(B),'B'+str(rep),con)
    sql.to_sql(Series(1/sigma_inv2),'Sigma'+str(rep),con)
    sql.to_sql(DataFrame(phi),'phi'+str(rep),con)
    sql.to_sql(Series(delta),'delta'+str(rep),con)
    sql.to_sql(Series(np.array([a1,a2])),'a'+str(rep),con)
    
con.close()

0
Optimization terminated successfully.
         Current function value: 0.928378
         Iterations: 29
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: 0.584902
         Iterations: 29
         Function evaluations: 58
1
Optimization terminated successfully.
         Current function value: -0.041554
         Iterations: 29
         Function evaluations: 60
Optimization terminated successfully.
         Current function value: -0.923720
         Iterations: 27
         Function evaluations: 55
2
Optimization terminated successfully.
         Current function value: -0.313879
         Iterations: 31
         Function evaluations: 65
Optimization terminated successfully.
         Current function value: -1.104115
         Iterations: 26
         Function evaluations: 53
3
Optimization terminated successfully.
         Current function value: -0.285098
         Iterations: 27
         Function evaluations: 56
Optimization terminat

30
Optimization terminated successfully.
         Current function value: -0.082953
         Iterations: 29
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: -2.696852
         Iterations: 29
         Function evaluations: 59
31
Optimization terminated successfully.
         Current function value: -0.034766
         Iterations: 30
         Function evaluations: 62
Optimization terminated successfully.
         Current function value: -2.772252
         Iterations: 30
         Function evaluations: 60
32
Optimization terminated successfully.
         Current function value: -0.020969
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -2.825068
         Iterations: 29
         Function evaluations: 60
33
Optimization terminated successfully.
         Current function value: -0.032517
         Iterations: 28
         Function evaluations: 58
Optimization te

60
Optimization terminated successfully.
         Current function value: -0.153287
         Iterations: 26
         Function evaluations: 54
Optimization terminated successfully.
         Current function value: -4.804458
         Iterations: 27
         Function evaluations: 56
61
Optimization terminated successfully.
         Current function value: -0.208417
         Iterations: 27
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: -4.798111
         Iterations: 26
         Function evaluations: 54
62
Optimization terminated successfully.
         Current function value: -0.243552
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -4.844635
         Iterations: 28
         Function evaluations: 58
63
Optimization terminated successfully.
         Current function value: -0.209456
         Iterations: 25
         Function evaluations: 52
Optimization te

90
Optimization terminated successfully.
         Current function value: -0.300350
         Iterations: 30
         Function evaluations: 62
Optimization terminated successfully.
         Current function value: -5.603655
         Iterations: 29
         Function evaluations: 60
91
Optimization terminated successfully.
         Current function value: -0.374896
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -5.756062
         Iterations: 31
         Function evaluations: 64
92
Optimization terminated successfully.
         Current function value: -0.378666
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -5.610366
         Iterations: 28
         Function evaluations: 58
93
Optimization terminated successfully.
         Current function value: -0.315342
         Iterations: 28
         Function evaluations: 56
Optimization te

120
Optimization terminated successfully.
         Current function value: -0.155636
         Iterations: 26
         Function evaluations: 53
Optimization terminated successfully.
         Current function value: -5.921076
         Iterations: 29
         Function evaluations: 59
121
Optimization terminated successfully.
         Current function value: -0.207048
         Iterations: 32
         Function evaluations: 65
Optimization terminated successfully.
         Current function value: -5.872102
         Iterations: 33
         Function evaluations: 67
122
Optimization terminated successfully.
         Current function value: -0.261852
         Iterations: 27
         Function evaluations: 55
Optimization terminated successfully.
         Current function value: -5.830140
         Iterations: 30
         Function evaluations: 61
123
Optimization terminated successfully.
         Current function value: -0.221038
         Iterations: 29
         Function evaluations: 59
Optimizatio

...

2873
Optimization terminated successfully.
         Current function value: 0.126734
         Iterations: 25
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: -8.708104
         Iterations: 33
         Function evaluations: 68
2874
Optimization terminated successfully.
         Current function value: 0.201148
         Iterations: 31
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -9.005731
         Iterations: 28
         Function evaluations: 59
2875
Optimization terminated successfully.
         Current function value: 0.230355
         Iterations: 28
         Function evaluations: 58
Optimization terminated successfully.
         Current function value: -8.958209
         Iterations: 32
         Function evaluations: 65
2876
Optimization terminated successfully.
         Current function value: 0.261465
         Iterations: 29
         Function evaluations: 59
Optimizatio

2903
Optimization terminated successfully.
         Current function value: 0.299598
         Iterations: 28
         Function evaluations: 58
Optimization terminated successfully.
         Current function value: -8.029406
         Iterations: 31
         Function evaluations: 65
2904
Optimization terminated successfully.
         Current function value: 0.294115
         Iterations: 31
         Function evaluations: 64
Optimization terminated successfully.
         Current function value: -8.186724
         Iterations: 28
         Function evaluations: 58
2905
Optimization terminated successfully.
         Current function value: 0.292075
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -8.190830
         Iterations: 34
         Function evaluations: 69
2906
Optimization terminated successfully.
         Current function value: 0.287357
         Iterations: 30
         Function evaluations: 63
Optimizatio

2933
Optimization terminated successfully.
         Current function value: 0.214769
         Iterations: 32
         Function evaluations: 65
Optimization terminated successfully.
         Current function value: -9.700756
         Iterations: 29
         Function evaluations: 59
2934
Optimization terminated successfully.
         Current function value: 0.181239
         Iterations: 30
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -9.544035
         Iterations: 31
         Function evaluations: 65
2935
Optimization terminated successfully.
         Current function value: 0.167441
         Iterations: 31
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -9.129923
         Iterations: 34
         Function evaluations: 70
2936
Optimization terminated successfully.
         Current function value: 0.189868
         Iterations: 27
         Function evaluations: 56
Optimizatio

2963
Optimization terminated successfully.
         Current function value: 0.013134
         Iterations: 32
         Function evaluations: 66
Optimization terminated successfully.
         Current function value: -8.413858
         Iterations: 35
         Function evaluations: 72
2964
Optimization terminated successfully.
         Current function value: 0.004637
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -8.332080
         Iterations: 28
         Function evaluations: 57
2965
Optimization terminated successfully.
         Current function value: -0.030713
         Iterations: 31
         Function evaluations: 64
Optimization terminated successfully.
         Current function value: -8.559213
         Iterations: 34
         Function evaluations: 70
2966
Optimization terminated successfully.
         Current function value: -0.028726
         Iterations: 28
         Function evaluations: 58
Optimizat

2993
Optimization terminated successfully.
         Current function value: 0.138160
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -8.960461
         Iterations: 33
         Function evaluations: 68
2994
Optimization terminated successfully.
         Current function value: 0.169052
         Iterations: 31
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -8.930895
         Iterations: 27
         Function evaluations: 57
2995
Optimization terminated successfully.
         Current function value: 0.208805
         Iterations: 31
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -8.943274
         Iterations: 28
         Function evaluations: 59
2996
Optimization terminated successfully.
         Current function value: 0.152523
         Iterations: 28
         Function evaluations: 58
Optimizatio

In [6]:
B=B_start.copy()
Omega=Omega_start.copy()
sigma_inv2=np.ones(G)
phi=np.ones((G,K))
a1=2
a2=2
delta=np.ones(K)
delta[0]=1
tau=np.array([np.prod(delta[:k+1]) for k in range(K)])

database=name+'-Dunsonofm'
con=sqlite3.connect('D:\\Python Database\\FM\\'+database)

for rep in range(3000):
    print(rep)
    
    update_Omega5()
    update_B()
    update_sigma()
    update_phi()
    update_a()
    update_delta()

    sql.to_sql(DataFrame(Omega),'Omega'+str(rep),con)
    sql.to_sql(DataFrame(B),'B'+str(rep),con)
    sql.to_sql(Series(1/sigma_inv2),'Sigma'+str(rep),con)
    sql.to_sql(DataFrame(phi),'phi'+str(rep),con)
    sql.to_sql(Series(delta),'delta'+str(rep),con)
    sql.to_sql(Series(np.array([a1,a2])),'a'+str(rep),con)
    
con.close()

0
Optimization terminated successfully.
         Current function value: 0.928378
         Iterations: 29
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: 0.584902
         Iterations: 29
         Function evaluations: 58
1
Optimization terminated successfully.
         Current function value: -0.269056
         Iterations: 29
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: -0.877206
         Iterations: 26
         Function evaluations: 52
2
Optimization terminated successfully.
         Current function value: -0.604888
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -1.605507
         Iterations: 31
         Function evaluations: 64
3
Optimization terminated successfully.
         Current function value: -0.580436
         Iterations: 25
         Function evaluations: 51
Optimization terminat

30
Optimization terminated successfully.
         Current function value: -0.323816
         Iterations: 27
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: -5.228954
         Iterations: 26
         Function evaluations: 54
31
Optimization terminated successfully.
         Current function value: -0.314812
         Iterations: 31
         Function evaluations: 64
Optimization terminated successfully.
         Current function value: -5.281780
         Iterations: 30
         Function evaluations: 61
32
Optimization terminated successfully.
         Current function value: -0.335567
         Iterations: 30
         Function evaluations: 62
Optimization terminated successfully.
         Current function value: -5.388392
         Iterations: 31
         Function evaluations: 63
33
Optimization terminated successfully.
         Current function value: -0.312761
         Iterations: 28
         Function evaluations: 57
Optimization te

60
Optimization terminated successfully.
         Current function value: -0.471435
         Iterations: 29
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: -4.931725
         Iterations: 28
         Function evaluations: 59
61
Optimization terminated successfully.
         Current function value: -0.520043
         Iterations: 26
         Function evaluations: 54
Optimization terminated successfully.
         Current function value: -4.805126
         Iterations: 26
         Function evaluations: 54
62
Optimization terminated successfully.
         Current function value: -0.538251
         Iterations: 29
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: -5.075825
         Iterations: 28
         Function evaluations: 58
63
Optimization terminated successfully.
         Current function value: -0.513714
         Iterations: 28
         Function evaluations: 58
Optimization te

90
Optimization terminated successfully.
         Current function value: -0.258048
         Iterations: 29
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -6.830945
         Iterations: 27
         Function evaluations: 56
91
Optimization terminated successfully.
         Current function value: -0.224616
         Iterations: 29
         Function evaluations: 60
Optimization terminated successfully.
         Current function value: -7.034926
         Iterations: 32
         Function evaluations: 65
92
Optimization terminated successfully.
         Current function value: -0.287307
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -7.205634
         Iterations: 26
         Function evaluations: 52
93
Optimization terminated successfully.
         Current function value: -0.338004
         Iterations: 31
         Function evaluations: 64
Optimization te

120
Optimization terminated successfully.
         Current function value: -0.328642
         Iterations: 28
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: -7.755418
         Iterations: 28
         Function evaluations: 57
121
Optimization terminated successfully.
         Current function value: -0.290149
         Iterations: 31
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -7.548816
         Iterations: 33
         Function evaluations: 68
122
Optimization terminated successfully.
         Current function value: -0.333877
         Iterations: 31
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -7.391451
         Iterations: 30
         Function evaluations: 63
123
Optimization terminated successfully.
         Current function value: -0.402639
         Iterations: 29
         Function evaluations: 60
Optimizatio

...

2882
Optimization terminated successfully.
         Current function value: -0.261315
         Iterations: 30
         Function evaluations: 62
Optimization terminated successfully.
         Current function value: -8.591931
         Iterations: 27
         Function evaluations: 56
2883
Optimization terminated successfully.
         Current function value: -0.204105
         Iterations: 30
         Function evaluations: 62
Optimization terminated successfully.
         Current function value: -8.533438
         Iterations: 29
         Function evaluations: 60
2884
Optimization terminated successfully.
         Current function value: -0.123128
         Iterations: 25
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: -8.866579
         Iterations: 31
         Function evaluations: 63
2885
Optimization terminated successfully.
         Current function value: -0.080714
         Iterations: 32
         Function evaluations: 67
Optimiz

2911
Optimization terminated successfully.
         Current function value: -0.307002
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -8.333234
         Iterations: 27
         Function evaluations: 57
2912
Optimization terminated successfully.
         Current function value: -0.339210
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -8.583298
         Iterations: 32
         Function evaluations: 66
2913
Optimization terminated successfully.
         Current function value: -0.404165
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -8.412163
         Iterations: 27
         Function evaluations: 54
2914
Optimization terminated successfully.
         Current function value: -0.443049
         Iterations: 26
         Function evaluations: 54
Optimiz

2940
Optimization terminated successfully.
         Current function value: -0.402532
         Iterations: 27
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: -8.674391
         Iterations: 29
         Function evaluations: 60
2941
Optimization terminated successfully.
         Current function value: -0.475076
         Iterations: 30
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -8.704181
         Iterations: 29
         Function evaluations: 60
2942
Optimization terminated successfully.
         Current function value: -0.513560
         Iterations: 31
         Function evaluations: 63
Optimization terminated successfully.
         Current function value: -8.590326
         Iterations: 29
         Function evaluations: 59
2943
Optimization terminated successfully.
         Current function value: -0.548747
         Iterations: 25
         Function evaluations: 51
Optimiz

2969
Optimization terminated successfully.
         Current function value: -0.352624
         Iterations: 28
         Function evaluations: 57
Optimization terminated successfully.
         Current function value: -8.958377
         Iterations: 31
         Function evaluations: 65
2970
Optimization terminated successfully.
         Current function value: -0.483725
         Iterations: 27
         Function evaluations: 55
Optimization terminated successfully.
         Current function value: -9.035952
         Iterations: 29
         Function evaluations: 60
2971
Optimization terminated successfully.
         Current function value: -0.471770
         Iterations: 26
         Function evaluations: 54
Optimization terminated successfully.
         Current function value: -8.819707
         Iterations: 32
         Function evaluations: 65
2972
Optimization terminated successfully.
         Current function value: -0.464636
         Iterations: 27
         Function evaluations: 56
Optimiz

2998
Optimization terminated successfully.
         Current function value: -0.351943
         Iterations: 30
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: -8.409838
         Iterations: 32
         Function evaluations: 66
2999
Optimization terminated successfully.
         Current function value: -0.282291
         Iterations: 27
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: -8.451619
         Iterations: 31
         Function evaluations: 64
